# Working RAdam

In [384]:
import numpy as np

# https://arxiv.org/pdf/1908.03265
# https://stats.stackexchange.com/questions/232741/why-is-it-important-to-include-a-bias-correction-term-for-the-adam-optimizer-for
class RAdam:
    """Rectified Adam (RAdam) optimizer.

    This optimizer dynamically adjusts the learning rate using a rectification term
    to stabilize the variance of the adaptive learning rate, especially in early training.

    Attributes:
        lr (float): Learning rate.
        betas (tuple): Coefficients for computing running averages of gradient and its square.
        eps (float): Term added to the denominator to improve numerical stability.
    """
    def __init__(self, lr=1e-3, betas=(0.9, 0.999), eps=1e-8):
        """Initialize the RAdam optimizer.

        Args:
            lr (float): Learning rate (default: 1e-3).
            betas (tuple): Coefficients for computing running averages of gradient and its square
                           (default: (0.9, 0.999)).
            eps (float): Term added to the denominator to improve numerical stability (default: 1e-8).

        Raises:
            ValueError: If `lr`, `betas`, or `eps` are invalid.
        """
        # Parameter validation
        if lr <= 0:
            raise ValueError("Learning rate (lr) must be positive.")
        if not (0.0 <= betas[0] < 1.0 and 0.0 <= betas[1] < 1.0):
            raise ValueError("Betas must be in the range [0, 1).")
        if eps <= 0:
            raise ValueError("Epsilon (eps) must be positive.")

        # Parameter initialization
        self.lr = lr    # Learning rate
        self.betas = betas  # Coefficients for moment estimates
        self.eps = eps  # Small constant for numerical stability

        # Initialize moment estimates (first and second moments)
        self.m = None  # First moment estimate (mean of gradients)
        self.v = None  # Second moment estimate (uncentered variance of gradients)

        # Asymptotic upper bound for the rectification term
        self.rho_inf = 2 / (1 - betas[1]) - 1

    def solve(self, objective_function, params, gradient_function, num_iterations=100):
        """Solve the optimization problem using RAdam optimizer.

        Args:
            objective_function (callable): The objective function to minimize.
            params (list of np.ndarray): List of parameters to optimize.
            gradient_function (callable): Function to compute gradients of the objective function.
            num_iterations (int): Number of optimization iterations (default: 100).

        Returns:
            list of np.ndarray: Optimized parameters.

        Raises:
            TypeError: If `objective_function`, `gradient_function` are invalid.
            ValueError: If `num_iterations` is invalid.
        """
        # Validate objective_function
        if not callable(objective_function):
            raise TypeError("objective_function must be a callable function.")

        # Validate gradient_function
        if not callable(gradient_function):
            raise TypeError("gradient_function must be a callable function.")

        # Validate num_iterations
        if not isinstance(num_iterations, int) or num_iterations <= 0:
            raise ValueError("num_iterations must be a positive integer.")

        # Initialize moment estimates
        self._initialize_moments(params)

        # Optimization loop
        for iteration in range(1, num_iterations + 1):
            # Compute gradients
            grads = list(gradient_function(*params))

            # Perform optimization step
            params = self.step(params, grads, iteration)

            # Print progress
            if (iteration + 1) % 10 == 0:
                loss = objective_function(*params)
                print(f"Iteration {iteration + 1}: x = {params[0][0]:.4f}, y = {params[1][0]:.4f}, Loss = {loss[0]:.4f}")

        return params

    def step(self, params, grads, t):
        """Perform a single optimization step using RAdam.

        Args:
            params (list of np.ndarray): Current parameters.
            grads (list of np.ndarray): Gradients of the objective function with respect to the parameters.
            t (int): Current iteration number (used for bias correction).

        Returns:
            list of np.ndarray: Updated parameters.

        Raises:
            AssertionError: If parameter and gradient shapes do not match.
        """
        # Ensure we have gradient for each parameter (shapes are equal)
        assert len(params) == len(grads), "Parameter and gradient shapes must match."

        # Lazy initialization of moment estimates
        if self.m is None or self.v is None:
            self._initialize_moments(params)

        for i, (param, grad) in enumerate(zip(params, grads)):
            # Ensure gradients and parameters have the same shape (needed for multidimensional parameters such as matricies)
            assert param.shape == grad.shape, "Parameter and gradient shapes must match."

            # Update biased first moment estimate
            self.m[i] = self.betas[0] * self.m[i] + (1 - self.betas[0]) * grad

            # Update biased second moment estimate
            self.v[i] = self.betas[1] * self.v[i] + (1 - self.betas[1]) * (grad ** 2)

            # Compute bias-corrected first moment estimate
            m_hat = self.m[i] / (1 - self.betas[0] ** t)

            # Rectification term
            rho_t = self.rho_inf - 2 * t * (self.betas[1] ** t) / (1 - self.betas[1] ** t)

            if rho_t > 4:
                # Rectified update
                l_t = np.sqrt( (1 - self.betas[1]**t) / (self.v[i] + self.eps) )
                r_t = np.sqrt(
                    ((rho_t - 4) * (rho_t - 2) * self.rho_inf) / ((self.rho_inf - 4) * (self.rho_inf - 2) * rho_t)
                )
                param_update = r_t * m_hat * l_t
            else:
                # Unrectified update (similar to Adam)
                param_update = m_hat

            # Update parameter
            param -= self.lr * param_update

        return params

    def _initialize_moments(self, params):
        """
        Initialize moment estimates (self.m and self.v) to zero vectors.

        Args:
            params (list of np.ndarray): Current parameters.
        """
        self.m = [np.zeros_like(p) for p in params]  # First moment estimate
        self.v = [np.zeros_like(p) for p in params]  # Second moment estimate

# Example usage with an arbitrary objective function
if __name__ == "__main__":
    # Define an example objective function (e.g., f(x, y) = x^2 + y^2)
    def objective_function(x, y, z):
        return (x-3)**2 + (y-2)**2 + (z-3)**2

    # Define the gradient of the objective function
    def gradient(x, y, z):
        return 2 * (x-3), 2 * (y-2), 2*(z-3)

    # Initialize parameters (e.g., x and y)
    params = [np.array([1.0]), np.array([1.0]), np.array([1.0])]  # Example: x = 1.0, y = 2.0

    # Initialize RAdam optimizer
    optimizer = RAdam(lr=0.1)
    result = optimizer.solve(objective_function, params, gradient)

    # Final optimized parameters
    print("\nOptimized parameters:")
    print(f"x = {result[0][0]:.4f}, y = {result[1][0]:.4f}, z = {result[2][0]:.4f}")

Iteration 10: x = 2.3675, y = 1.6903, Loss = 0.8959
Iteration 20: x = 2.4194, y = 1.7414, Loss = 0.7412
Iteration 30: x = 2.4928, y = 1.8102, Loss = 0.5505
Iteration 40: x = 2.5798, y = 1.8837, Loss = 0.3667
Iteration 50: x = 2.6723, y = 1.9484, Loss = 0.2174
Iteration 60: x = 2.7625, y = 1.9933, Loss = 0.1128
Iteration 70: x = 2.8432, y = 2.0144, Loss = 0.0494
Iteration 80: x = 2.9089, y = 2.0161, Loss = 0.0168
Iteration 90: x = 2.9568, y = 2.0083, Loss = 0.0038
Iteration 100: x = 2.9872, y = 2.0005, Loss = 0.0003

Optimized parameters:
x = 2.9893, y = 1.9999, z = 2.9893


## Pytorch's RAdam

In [ ]:
import torch
import torch.optim as optim

# Define an example objective function (e.g., f(x, y) = x^2 + y^2)
def objective_function(x, y):
    return (x-2)**2 + (y-2)**2 + (z-3)**2

# Initialize parameters (e.g., x and y)
x = torch.tensor([1.0], requires_grad=True)  # Example: x = 1.0
y = torch.tensor([1.0], requires_grad=True)  # Example: y = 2.0
z = torch.tensor([1.0], requires_grad=True)  # Example: y = 2.0

# Initialize RAdam optimizer
optimizer = optim.RAdam([x, y, z], lr=0.1)

# Optimization loop
num_iterations = 100
for iteration in range(num_iterations):
    # Zero the gradients
    optimizer.zero_grad()

    # Compute the objective function
    loss = objective_function(x, y)

    # Compute gradients
    loss.backward()

    # Perform optimization step
    optimizer.step()

    # Print progress
    if (iteration + 1) % 10 == 0:
        print(f"Iteration {iteration + 1}: x = {x.item():.4f}, y = {y.item():.4f}, Loss = {loss.item():.4f}")

# Final optimized parameters
print("\nOptimized parameters:")
print(f"x = {x.item():.4f}, y = {y.item():.4f}, z = {z.item():.4f}")

Iteration 10: x = 1.8125, y = 1.8125, Loss = 0.2270
Iteration 20: x = 1.8553, y = 1.8553, Loss = 0.1674
Iteration 30: x = 1.9064, y = 1.9064, Loss = 0.1061
Iteration 40: x = 1.9555, y = 1.9555, Loss = 0.0578
Iteration 50: x = 1.9923, y = 1.9923, Loss = 0.0273
Iteration 60: x = 2.0110, y = 2.0110, Loss = 0.0110
Iteration 70: x = 2.0132, y = 2.0132, Loss = 0.0034
Iteration 80: x = 2.0067, y = 2.0067, Loss = 0.0005
Iteration 90: x = 1.9999, y = 1.9999, Loss = 0.0000
Iteration 100: x = 1.9974, y = 1.9974, Loss = 0.0001

Optimized parameters:
x = 1.9974, y = 1.9974, z = 3.0078


## Barrier function construction (SYMPY)

### First step

In [ ]:
import sympy as sp

# Define symbolic variables
x1, x2 = sp.symbols('x1 x2')
mu = sp.symbols('mu', positive=True)

# Define the original objective function f(x)
f = x1**2 + x2**2  # Example objective function

# Define the non-linear inequality constraints g_i(x) <= 0
g1 = x1 + x2 - 1  # Example constraint 1
g2 = -x1 + x2 - 1  # Example constraint 2

# Define the barrier function phi(x)
phi = -sp.log(-g1) - sp.log(-g2)

# Define the new objective function F(x, mu) with the barrier method applied
F = f + mu * phi

# Display the new objective function
F

mu*(-log(-x1 - x2 + 1) - log(x1 - x2 + 1)) + x1**2 + x2**2

### Second step

In [ ]:
import sympy as sp

def create_objective_with_barrier(f, constraints, mu):
    """
    Creates a new objective function with the barrier method applied.

    Parameters:
        f (sympy expression): The original objective function.
        constraints (list of sympy expressions): List of non-linear inequality constraints (g_i(x) <= 0).
        mu (sympy symbol or float): Barrier parameter.

    Returns:
        sympy expression: New objective function with the barrier method applied.
    """
    # Define the barrier function phi(x)
    phi = -sum(sp.log(-g) for g in constraints)

    # Create the new objective function F(x, mu) = f(x) + mu * phi(x)
    F = f + mu * phi

    return F

# Example usage
if __name__ == "__main__":
    # Define symbolic variables
    x1, x2 = sp.symbols('x1 x2')
    mu = sp.symbols('mu', positive=True)  # Barrier parameter

    # User-defined objective function
    f = x1**2 + x2**2  # Example: f(x1, x2) = x1^2 + x2^2

    # User-defined inequality constraints (g_i(x) <= 0)
    constraints = [
        x1 + x2 - 1,  # g1(x1, x2) = x1 + x2 - 1 <= 0
        -x1 + x2 - 1  # g2(x1, x2) = -x1 + x2 - 1 <= 0
    ]

    # Create the new objective function with the barrier method applied
    F = create_objective_with_barrier(f, constraints, mu)

    # Display the new objective function
    print("Original Objective Function:")
    print(f)
    print("\nConstraints:")
    for i, g in enumerate(constraints):
        print(f"g{i+1}(x) = {g} <= 0")
    print("\nNew Objective Function with Barrier Method Applied:")
    print(F)

Original Objective Function:
x1**2 + x2**2

Constraints:
g1(x) = x1 + x2 - 1 <= 0
g2(x) = -x1 + x2 - 1 <= 0

New Objective Function with Barrier Method Applied:
mu*(-log(-x1 - x2 + 1) - log(x1 - x2 + 1)) + x1**2 + x2**2


### Adding gradient calculation

In [ ]:
def gradient_function(F, variables):
    """
    Calculates the gradient of the function F with respect to the given variables.

    Parameters:
        F (sympy expression): The function to compute the gradient for.
        variables (list of sympy symbols): List of variables to differentiate with respect to.

    Returns:
        list of sympy expressions: The gradient of F.
    """
    gradient = [sp.diff(F, var) for var in variables]
    return gradient

if __name__ == '__main__':
    # Calculate the gradient of F with respect to x1 and x2
    variables = [x1, x2]
    gradient = gradient_function(F, variables)

    # Display the gradient
    print("Gradient of the New Objective Function:")
    for i, grad in enumerate(gradient):
        print(f"∂F/∂{variables[i]} = {grad}")

Gradient of the New Objective Function:
∂F/∂x1 = mu*(-1/(x1 - x2 + 1) + 1/(-x1 - x2 + 1)) + 2*x1
∂F/∂x2 = mu*(1/(x1 - x2 + 1) + 1/(-x1 - x2 + 1)) + 2*x2


## Solve constrained problem (inequality constraints only)

In [ ]:
def solve_constrained_optimization(objective_function, constraints, initial_guess, mu=1.0, lr=1e-2, num_iterations=100):
    # Step 1: Extract variables from the objective function
    variables = list(f.free_symbols)
    variables.sort(key=lambda var: var.name)  # Sort variables for consistency

    # Step 2: Create the new objective function with the barrier method
    F = create_objective_with_barrier(objective_function, constraints, mu)

    # Step 3: Compute the gradient of F
    gradient = gradient_function(F, variables)

    # Step 4: Convert symbolic expressions to numerical functions
    F_numeric = sp.lambdify(variables, F, 'numpy')
    gradient_numeric = sp.lambdify(variables, gradient, 'numpy')

    # Step 5: Define the objective and gradient functions for RAdam
    def objective_function(*params):
        return F_numeric(*params)

    def gradient_function_wrapper(*params):
        grads = gradient_numeric(*params)
        return [np.array(grad) for grad in grads]

    # Step 6: forming RAdam parameters
    params = [np.array([x]) for x in initial_guess]
    optimizer = RAdam(lr=lr)

    # Solve the optimization problem
    optimized_params = optimizer.solve(objective_function, params, gradient_function_wrapper, num_iterations)

    # Return the optimized parameters
    return [p[0] for p in optimized_params]

# Example usage
if __name__ == "__main__":
    # Define symbolic variables
    x1, x2 = sp.symbols('x1 x2')

    # User-defined objective function
    f = x1**2 + x2**2  # Example: f(x1, x2) = x1^2 + x2^2

    # User-defined inequality constraints (g_i(x) <= 0)
    constraints = [
        x1 + x2 - 1,  # g1(x1, x2) = x1 + x2 - 1 <= 0
        -x1 + x2 - 1  # g2(x1, x2) = -x1 + x2 - 1 <= 0
    ]

    # Initial guess for the variables (!!!the initual guess point must be in feasible region!!!)
    initial_guess = [-1.0, -1.0]

    # Solve the constrained optimization problem
    optimized_params = solve_constrained_optimization(f, constraints, initial_guess, mu=1.0, lr=1e-2, num_iterations=100)

    # Print the optimized parameters
    print("Optimized Parameters:")
    print(f"x1 = {optimized_params[0]:.4f}, x2 = {optimized_params[1]:.4f}")

[-1.]
0.01 [-2.66666667]
[-1.]
0.01 [-0.66666667]
[-0.97333333]
0.01 [-2.62630535]
[-0.99333333]
0.01 [-0.66799782]
[-0.94707028]
0.01 [-2.5853074]
[-0.98665336]
0.01 [-0.6689281]
[-0.92121721]
0.01 [-2.54371178]
[-0.97996407]
0.01 [-0.66945152]
[-0.89578009]
0.01 [-0.01719123]
[-0.97326956]
0.01 [-0.01731551]
[-0.89560818]
0.01 [-0.02560114]
[-0.9730964]
0.01 [-0.02582601]
[-0.89535216]
0.01 [-0.0324269]
[-0.97283814]
0.01 [-0.03274267]
[-0.8950279]
0.01 [-0.03834316]
[-0.97251072]
0.01 [-0.03873868]
[-0.89464446]
0.01 [-0.04363473]
[-0.97212333]
0.01 [-0.04409881]
Iteration 10: x = -0.8942, y = -0.9717, Loss = 0.6163
[-0.89420812]
0.01 [-0.0484601]
[-0.97168234]
0.01 [-0.04898192]
[-0.89372352]
0.01 [-0.05291974]
[-0.97119252]
0.01 [-0.05348896]
[-0.89319432]
0.01 [-0.05708253]
[-0.97065763]
0.01 [-0.05768918]
[-0.89262349]
0.01 [-0.06099819]
[-0.97008074]
0.01 [-0.06163262]
[-0.89201351]
0.01 [-0.06470401]
[-0.96946442]
0.01 [-0.06535679]
[-0.89136647]
0.01 [-0.06822878]
[-0.9688108

# Forming Tools class

In [ ]:
import sympy as sp

class BarrierMethodTools:
    """A static class containing utility functions for the barrier method."""

    @staticmethod
    def create_barrier_function(constraints):
        """
        Creates the barrier function phi(x) for the given constraints using the property:
        sum(log(g_i)) = log(product(g_i)).

        Args:
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).

        Returns:
            sympy expression: The barrier function phi(x).
        """
        # Compute the product of the negative constraints
        product_of_constraints = sp.prod(-g for g in constraints)

        # Barrier function: phi(x) = -log(product_of_constraints)
        phi = -sp.log(product_of_constraints)
        return phi

    @staticmethod
    def create_objective_with_barrier(f, constraints, mu):
        """
        Creates a new objective function with the barrier method applied.

        Args:
            f (sympy expression): The original objective function.
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
            mu (sympy symbol or float): Barrier parameter.

        Returns:
            sympy expression: The new objective function F(x, mu).
        """
        # Create the barrier function
        phi = BarrierMethodTools.create_barrier_function(constraints)

        # New objective function: F(x, mu) = f(x) + mu * phi(x)
        F = f + mu * phi
        return F

    @staticmethod
    def gradient_function(F, variables):
        """
        Calculates the gradient of the function F with respect to the given variables.

        Args:
            F (sympy expression): The function to compute the gradient for.
            variables (list of sympy symbols): List of variables to differentiate with respect to.

        Returns:
            list of sympy expressions: The gradient of F.
        """
        gradient = [sp.diff(F, var) for var in variables]
        return gradient

In [ ]:
import numpy as np


# +++TODO #1: adaptive barrier parameter
# TODO #2: check the initial guess is in the feasible reagion
# TODO #3: error checks
def solve_constrained_optimization(f, constraints, initial_guess, mu=1.0, lr=1e-2, num_iterations=100):
    """
    Solve a constrained optimization problem using the barrier method and RAdam optimizer.

    Args:
        f (sympy expression): The objective function to minimize.
        constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
        initial_guess (list of float): Initial guess for the variables.
        mu (float): Barrier parameter (default: 1.0).
        lr (float): Learning rate for RAdam (default: 1e-2).
        num_iterations (int): Number of optimization iterations (default: 100).

    Returns:
        list of float: Optimized parameters.
    """
    # Extract variables from the objective function only
    variables = list(f.free_symbols)
    variables.sort(key=lambda var: var.name)  # Sort variables for consistency

    # Create the new objective function with the barrier method (symbolic)
    F_sym = BarrierMethodTools.create_objective_with_barrier(f, constraints, mu)

    # Compute the gradient of F (symbolic)
    gradient_sym = BarrierMethodTools.gradient_function(F_sym, variables)

    # Convert symbolic expressions to numerical functions
    F_numeric = sp.lambdify(variables, F_sym, 'numpy')
    gradient_numeric = sp.lambdify(variables, gradient_sym, 'numpy')

    # Define the objective and gradient functions for RAdam
    def objective_function_wrapper(*params):
        return F_numeric(*params)

    def gradient_function_wrapper(*params):
        grads = gradient_numeric(*params)
        return [np.array(grad) for grad in grads]

    # Initialize parameters
    params = [np.array([x]) for x in initial_guess]
    print(params)

    # Initialize RAdam optimizer
    optimizer = RAdam(lr=lr)

    # Solve the optimization problem
    optimized_params = optimizer.solve(objective_function_wrapper, params, gradient_function_wrapper, num_iterations)

    # Return the optimized parameters
    return [p[0] for p in optimized_params]


if __name__ == '__main__':
    # Define symbolic variables
    x1, x2 = sp.symbols('x1 x2')

    # User-defined objective function
    f = x1**2 + x2**2  # Example: f(x1, x2) = x1^2 + x2^2

    # User-defined inequality constraints (g_i(x) <= 0)
    constraints = [
        x1**2-x2
        #x1 + x2 - 1,  # g1(x1, x2) = x1 + x2 - 1 <= 0
        #-x1 + x2 - 1  # g2(x1, x2) = -x1 + x2 - 1 <= 0
    ]

    # Initial guess for the variables
    initial_guess = [0.0, 1.0]

    # Solve the constrained optimization problem
    optimized_params = solve_constrained_optimization(f, constraints, initial_guess,
                                                      mu=0.001, lr=1e-2, num_iterations=100)

    # Print the optimized parameters
    print("Optimized Parameters:")
    print(f"x1 = {optimized_params[0]:.4f}, x2 = {optimized_params[1]:.4f}")

[array([0.]), array([1.])]
Iteration 10: x = 0.0000, y = 0.9197, Loss = 0.8460
Iteration 20: x = 0.0000, y = 0.9131, Loss = 0.8339
Iteration 30: x = 0.0000, y = 0.9036, Loss = 0.8166
Iteration 40: x = 0.0000, y = 0.8918, Loss = 0.7954
Iteration 50: x = 0.0000, y = 0.8782, Loss = 0.7713
Iteration 60: x = 0.0000, y = 0.8630, Loss = 0.7450
Iteration 70: x = 0.0000, y = 0.8466, Loss = 0.7168
Iteration 80: x = 0.0000, y = 0.8289, Loss = 0.6873
Iteration 90: x = 0.0000, y = 0.8103, Loss = 0.6568
Iteration 100: x = 0.0000, y = 0.7908, Loss = 0.6256
Optimized Parameters:
x1 = 0.0000, x2 = 0.7888


In [ ]:
f.evalf(subs={x1: -0.7630, x2: -0.850})

1.30466900000000

In [ ]:
f.evalf(subs={x1: 0.5, x2: 0.5})

0.500000000000000

# Adaptive barrier parameter

In [ ]:
# as mu approaches zero, the closer to the original function we
def solve_constrained_optimization(f, constraints, initial_guess, mu_0=1.0, lr=1e-2, num_iterations=100, mu_decay=0.9):
    """
    Solve a constrained optimization problem using the barrier method and RAdam optimizer.

    Args:
        f (sympy expression): The objective function to minimize.
        constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
        initial_guess (list or np.ndarray): Initial guess for the variables.
        mu_0 (float): Initial barrier parameter (default: 1.0).
        lr (float): Learning rate for RAdam (default: 1e-2).
        num_iterations (int): Number of optimization iterations (default: 100).
        mu_decay (float): Decay factor for the barrier parameter (default: 0.9).

    Returns:
        list of float: Optimized parameters.
    """
    # Extract variables from the objective function only
    variables = list(f.free_symbols)
    variables.sort(key=lambda var: var.name)  # Sort variables for consistency

    # Initialize the barrier parameter
    mu = mu_0

    # Initialize RAdam optimizer
    optimizer = RAdam(lr=lr)

    # Convert initial_guess to a list of np.array objects
    params = [np.array([x]) for x in initial_guess]

    # Optimization loop
    for iteration in range(1, num_iterations + 1):
        # Create the new objective function with the current barrier parameter
        F_sym = BarrierMethodTools.create_objective_with_barrier(f, constraints, mu)

        # Compute the gradient of F (symbolic)
        gradient_sym = BarrierMethodTools.gradient_function(F_sym, variables)

        # Convert symbolic expressions to numerical functions
        F_numeric = sp.lambdify(variables, F_sym, 'numpy')
        gradient_numeric = sp.lambdify(variables, gradient_sym, 'numpy')

        # Define the objective and gradient functions for RAdam
        def objective_function(*params):
            return F_numeric(*params)

        def gradient_function_wrapper(*params):
            grads = gradient_numeric(*params)
            return [np.array(grad) for grad in grads]

        # Perform one optimization step
        params = optimizer.step(params, gradient_function_wrapper(*params), iteration)

        # Update the barrier parameter
        mu *= mu_decay

        # Print progress
        if (iteration + 1) % 10 == 0:
            loss = objective_function(*params)
            print(f"Iteration {iteration + 1}: Parameters = {[p[0] for p in params]}, Loss = {loss[0]:.4f}, mu = {mu:.4f}")

    # Return the optimized parameters as a list of floats
    return [p[0] for p in params]

if __name__ == '__main__':
    # Define symbolic variables
    x1, x2 = sp.symbols('x1 x2')

    # User-defined objective function
    f = x1**2 + x2**2  # Example: f(x1, x2) = x1^2 + x2^2

    # User-defined inequality constraints (g_i(x) <= 0)
    constraints = [
        x1**2-x2
        #x1 + x2 - 1,  # g1(x1, x2) = x1 + x2 - 1 <= 0
        #-x1 + x2 - 1  # g2(x1, x2) = -x1 + x2 - 1 <= 0
    ]

    # Initial guess for the variables
    initial_guess = [0.0, 1.0]

    # Solve the constrained optimization problem
    optimized_params = solve_constrained_optimization(
        f, constraints, initial_guess, mu_0=1.0, lr=1e-2, num_iterations=100, mu_decay=0.9
    )

    # Print the optimized parameters
    print("Optimized Parameters:")
    print(f"x1 = {optimized_params[0]:.4f}, x2 = {optimized_params[1]:.4f}")

Iteration 10: Parameters = [0.0, 0.9562350933627074], Loss = 0.9336, mu = 0.3874
Iteration 20: Parameters = [0.0, 0.9492441893354145], Loss = 0.9089, mu = 0.1351
Iteration 30: Parameters = [0.0, 0.9388080937151324], Loss = 0.8847, mu = 0.0471
Iteration 40: Parameters = [0.0, 0.9258240456220205], Loss = 0.8586, mu = 0.0164
Iteration 50: Parameters = [0.0, 0.9108814209386893], Loss = 0.8303, mu = 0.0057
Iteration 60: Parameters = [0.0, 0.8943843398221092], Loss = 0.8002, mu = 0.0020
Iteration 70: Parameters = [0.0, 0.8766101310457759], Loss = 0.7685, mu = 0.0007
Iteration 80: Parameters = [0.0, 0.8577562971042157], Loss = 0.7358, mu = 0.0002
Iteration 90: Parameters = [0.0, 0.8379730793690617], Loss = 0.7022, mu = 0.0001
Iteration 100: Parameters = [0.0, 0.8173831550179095], Loss = 0.6681, mu = 0.0000
Optimized Parameters:
x1 = 0.0000, x2 = 0.8153


# Adaptive barrier method as black-box interface to Barrier Tools

In [402]:
import sympy as sp
import numpy as np

class BarrierMethodTools:
    """A static class containing utility functions for the barrier method."""

    @staticmethod
    def create_barrier_function(constraints, epsilon=1e-6):
        """
        Creates the barrier function phi(x) for the given constraints using the logarithm of the product.

        Args:
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
            epsilon (float): Small offset to ensure constraints are strictly less than zero.

        Returns:
            sympy expression: The barrier function phi(x).
        """
        # Add a small offset to the constraints to avoid g_i(x) = 0
        offset_constraints = [g - epsilon for g in constraints]

        # Barrier function: phi(x) = -log(product_of_constraints)
        product_of_constraints = sp.prod(-g for g in offset_constraints)
        phi = -sp.log(product_of_constraints)
        return phi

    @staticmethod
    def create_objective_with_barrier(f, constraints, mu):
        """
        Creates a new objective function with the barrier method applied.

        Args:
            f (sympy expression): The original objective function.
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
            mu (sympy symbol or float): Barrier parameter.

        Returns:
            sympy expression: The new objective function F(x, mu).
        """
        # Create the barrier function
        phi = BarrierMethodTools.create_barrier_function(constraints)

        # New objective function: F(x, mu) = f(x) + mu * phi(x)
        F = f + mu * phi
        return F

    @staticmethod
    def gradient_function(F, variables):
        """
        Calculates the gradient of the function F with respect to the given variables.

        Args:
            F (sympy expression): The function to compute the gradient for.
            variables (list of sympy symbols): List of variables to differentiate with respect to.

        Returns:
            list of sympy expressions: The gradient of F.
        """
        gradient = [sp.diff(F, var) for var in variables]
        return gradient

    @staticmethod
    def is_feasible(constraints, variables, point, epsilon=1e-6):
        """
        Check if a point is feasible (satisfies all constraints).

        Args:
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
            variables (list of sympy symbols): List of variables in the constraints.
            point (list or np.ndarray): The point to check.
            epsilon (float): Tolerance for feasibility (default: 1e-6).

        Returns:
            bool: True if the point is feasible, False otherwise.
        """
        # Convert constraints to numerical functions
        constraint_funcs = [sp.lambdify(variables, g, 'numpy') for g in constraints]

        # Evaluate constraints at the point
        for g_func in constraint_funcs:
            if g_func(*point) >= -epsilon:
                return False  # Constraint violated

        return True  # All constraints satisfied

    @staticmethod
    def find_feasible_point(constraints, variables, initial_guess, optimizer, max_iterations=1000):
        """
        Find a feasible point by minimizing the constraint violations.

        Args:
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
            variables (list of sympy symbols): List of variables in the constraints.
            initial_guess (list or np.ndarray): Initial guess for the variables.
            optimizer: The optimizer to use (e.g., RAdam).
            max_iterations (int): Maximum number of iterations to find a feasible point (default: 1000).

        Returns:
            list of float: A feasible point.
        """
        # Define the objective function as the sum of squared constraint violations
        violation = sum(sp.Max(0, g)**2 for g in constraints)
        violation_func = sp.lambdify(variables, violation, 'numpy')

        # Define the gradient of the violation function
        gradient_violation = [sp.diff(violation, var) for var in variables]
        gradient_violation_func = sp.lambdify(variables, gradient_violation, 'numpy')

        # Convert initial_guess to a list of np.array objects
        params = [np.array([x]) for x in initial_guess]

        # Minimize the violation function
        for iteration in range(1, max_iterations + 1):
            # Extract scalar values from params
            param_values = [p[0] for p in params]

            # Compute gradients
            grads = np.array(gradient_violation_func(*param_values)).reshape(-1, 1)  # Ensure grads is a single array

            # Perform optimization step
            params = optimizer.step(params, grads, iteration)

            # Check if the point is feasible
            if BarrierMethodTools.is_feasible(constraints, variables, [p[0] for p in params]):
                return [p[0] for p in params]

        raise ValueError("Unable to find a feasible point within the maximum number of iterations.")

    @staticmethod
    def optimize_with_adaptive_barrier(f, constraints, initial_guess, optimizer, mu_0=1.0, num_iterations=100, mu_decay=0.9):
        """
        Solve a constrained optimization problem using the barrier method and a given optimizer.

        Args:
            f (sympy expression): The objective function to minimize.
            constraints (list of sympy expressions): List of inequality constraints (g_i(x) <= 0).
            initial_guess (list of floats or np.ndarray of floats): Initial guess for the variables.
            optimizer: The optimizer to use (e.g., RAdam).
            mu_0 (float): Initial barrier parameter (default: 1.0).
            num_iterations (int): Number of optimization iterations (default: 100).
            mu_decay (float): Decay factor for the barrier parameter (default: 0.9).

        Returns:
            list of float: Optimized parameters.
        """
        # Extract variables from the objective function only
        variables = list(f.free_symbols)
        variables.sort(key=lambda var: var.name)  # Sort variables for consistency

        # Check if the initial guess is feasible
        if not BarrierMethodTools.is_feasible(constraints, variables, initial_guess):
            raise ValueError("Initial guess is not in the feasible set!")

        # Initialize the barrier parameter
        mu = mu_0

        # Convert initial_guess to a list of np.array objects
        params = [np.array([x]) for x in initial_guess]

        # Optimization loop
        for iteration in range(1, num_iterations + 1):
            # Create the new objective function with the current barrier parameter
            F_sym = BarrierMethodTools.create_objective_with_barrier(f, constraints, mu)

            # Compute the gradient of F (symbolic)
            gradient_sym = BarrierMethodTools.gradient_function(F_sym, variables)

            # Convert symbolic expressions to numerical functions
            F_numeric = sp.lambdify(variables, F_sym, 'numpy')
            gradient_numeric = sp.lambdify(variables, gradient_sym, 'numpy')

            # Define the objective and gradient functions for the optimizer
            def objective_function(*params):
                return F_numeric(*params)

            def gradient_function_wrapper(*params):
                grads = gradient_numeric(*params)
                return [np.array(grad) for grad in grads]

            # Perform one optimization step
            params = optimizer.step(params, gradient_function_wrapper(*params), iteration)

            # Update the barrier parameter
            mu *= mu_decay

            # Print progress
            if (iteration + 1) % 10 == 0:
                loss = objective_function(*params)
                print(f"Iteration {iteration + 1}: Parameters = {[p[0] for p in params]}, Loss = {loss[0]:.4f}, mu = {mu:.4f}")

        # Return the optimized parameters as a list of floats
        return [p[0] for p in params]

if __name__ == '__main__':
    # Define symbolic variables
    x1, x2 = sp.symbols('x1 x2')

    # User-defined objective function
    f = x1**2 + x2**2  # Example: f(x1, x2) = x1^2 + x2^2

    # User-defined inequality constraints (g_i(x) <= 0)
    constraints = [
        x1**2 + x2,
        sp.exp(x1) + x2,
        sp.cos(x1)
        #x1 + x2 - 1,  # g1(x1, x2) = x1 + x2 - 1 <= 0
        #-x1 + x2 - 1  # g2(x1, x2) = -x1 + x2 - 1 <= 0
    ]

    # Initial guess for the variables
    initial_guess = [-2.0, -5.0]

    # Initialize RAdam optimizer
    optimizer = RAdam(lr=1e-2)

    # Solve the constrained optimization problem using BarrierMethodTools
    optimized_params = BarrierMethodTools.optimize_with_adaptive_barrier(
        f, constraints, initial_guess, optimizer, mu_0=1.0, num_iterations=100, mu_decay=0.9
    )

    # Print the optimized parameters
    print("Optimized Parameters:")
    print(f"x1 = {optimized_params[0]:.4f}, x2 = {optimized_params[1]:.4f}")

Iteration 10: Parameters = [-1.80063317411976, -4.645952343098949], Loss = 24.6723, mu = 0.3874
Iteration 20: Parameters = [-1.7944646894826297, -4.639265593251782], Loss = 24.6915, mu = 0.1351
Iteration 30: Parameters = [-1.7850249910943359, -4.629494133963085], Loss = 24.6021, mu = 0.0471
Iteration 40: Parameters = [-1.7730121586729897, -4.6173891781333465], Loss = 24.4588, mu = 0.0164
Iteration 50: Parameters = [-1.7589248413038039, -4.603360554067481], Loss = 24.2833, mu = 0.0057
Iteration 60: Parameters = [-1.743124934694012, -4.587675802629339], Loss = 24.0849, mu = 0.0020
Iteration 70: Parameters = [-1.725870723142038, -4.570523561563857], Loss = 23.8683, mu = 0.0007
Iteration 80: Parameters = [-1.7073490202925448, -4.552044616899687], Loss = 23.6362, mu = 0.0002
Iteration 90: Parameters = [-1.6877004213556264, -4.532349377029768], Loss = 23.3905, mu = 0.0001
Iteration 100: Parameters = [-1.667036273292761, -4.51152808033489], Loss = 23.1329, mu = 0.0000
Optimized Parameters:
x1